In [1]:
import pdal
import json
import geopandas
import pandas as pd
from shapely.geometry import Polygon, Point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
# loading json file
def read_json(json_path):
    try:
        with open(json_path) as js:
            json_obj = json.load(js)
        return json_obj

    except FileNotFoundError:
        print('File not found.')


In [6]:
dicti = read_json("pdal.json")

In [7]:
import geopandas as gpd
from shapely.geometry import Polygon
MINX, MINY, MAXX, MAXY = [-10436887.43333523, 5148706.389047224, -10436876.301386151, 5148721.349314567]
polygon_g = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_g])       
print(str(polygon.iloc[:,0][0]))

POLYGON ((-10436887.43333523 5148706.389047224, -10436887.43333523 5148721.349314567, -10436876.301386151 5148721.349314567, -10436876.301386151 5148706.389047224, -10436887.43333523 5148706.389047224))


In [8]:
import folium
m = folium.Map([41.918015, -93.756155],zoom_start=5, tiles='cartodbpositron')
folium.GeoJson(polygon).add_to(m)
folium.LatLngPopup().add_to(m)
m

In [9]:
dicti['pipeline'][0]['polygon'] = str(polygon.iloc[:,0][0])

In [10]:
pipe = pdal.Pipeline(json.dumps(dicti))

In [11]:
pipe.execute()

179

In [12]:
data = pd.read_csv("iowa.csv")
data2 = data.iloc[:,-3:]
data2.head()

,X,Y,Z
0,-93.756,41.918,310.03
1,-93.756,41.918,310.15
2,-93.756,41.918,310.14
3,-93.756,41.918,310.10
4,-93.756,41.918,309.90


In [13]:
def generate_geo(pipe, epsg):
    try:
        cloud_points = []
        elevations =[]
        geometry_points=[]
        for row in pipe.arrays[0]:
            lst = row.tolist()[-3:]
            cloud_points.append(lst)
            elevations.append(lst[2])
            point = Point(lst[0], lst[1])
            geometry_points.append(point)
        geodf = geopandas.GeoDataFrame(columns=["elevation", "geometry"])
        geodf['elevation'] = elevations
        geodf['geometry'] = geometry_points
        geodf = geodf.set_geometry("geometry")
        geodf.set_crs(epsg = epsg, inplace=True)
        return geodf
    except RuntimeError as e:
        self.logger.exception('fails to extract geo data frame')
        print(e)

In [14]:
generate_geo(pipe, 4326)

,elevation,geometry
0,310.03,POINT (-93.75606 41.91802)
1,310.15,POINT (-93.75611 41.91802)
2,310.14,POINT (-93.75613 41.91802)
3,310.10,POINT (-93.75614 41.91802)
4,309.90,POINT (-93.75615 41.91802)
...,...,...
174,310.21,POINT (-93.75613 41.91810)
175,310.22,POINT (-93.75609 41.91810)
176,310.07,POINT (-93.75609 41.91806)
177,310.05,POINT (-93.75608 41.91803)


In [2]:
from pyproj import Proj, transform

def convert_EPSG(fromT, lon, lat):
    P3857 = Proj(init='epsg:3857')
    P4326 = Proj(init='epsg:4326')
    if(fromT == 4326):
        input1 = P4326
        input2 = P3857
    else:
        input1=p3857
        input2=p4326
        
    return transform(input1,input2, lon, lat)

In [4]:
convert_EPSG(4326,-93.756055, 41.918115)

(-10436876.301386151, 5148721.349314567)

In [48]:
y

41.918015